In [31]:
# For more information refer - https://github.com/chamathpali/Fed-Goodreads

import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd

In [32]:
from pprint import pprint
import numpy as np
import random
from sklearn.feature_extraction.text import CountVectorizer
random.seed(0)
np.random.seed(0)

In [33]:
DIR = './'
def load_data(file_name, head = 500):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)
            
            # break if reaches the 100th line
            if (head is not None) and (count > head):
                break
    return data

In [62]:
reviews = load_data('goodreads_reviews_spoiler.json.gz', 200000)
# display(np.random.choice(reviews))
print(len(reviews))
# print(reviews[0])

random.seed(0)
np.random.seed(0)


all_data = []
for review in reviews:
    for sent in review["review_sentences"]:
        
        all_data.append({"user": review["user_id"], "sent": sent[1], "has_spoiler": sent[0] })

        
import pandas as pd         
all_reviews = pd.DataFrame(all_data)
all_reviews = all_reviews.sample(frac=1)

all_reviews.size
all_reviews.head(20)

200001


,user,sent,has_spoiler
1183653,58fe4880f1c5746d7311867093747775,They complain a lot and almost look stupid and...,0
2101183,abf8fe152026f9b2ac3581c956b5782b,I was drawn in to the story pretty easily and ...,0
84649,77008aaf6eb579f3c87a62fd59f936fd,I'm glad the writers didn't completely erase h...,0
2031516,e3fd6992ad541c59468adff19e8c94d5,People usually don't make the best decisions w...,0
1993814,2bea09c443d38d253711f23d7b331927,I'm surprised that there were some recycled id...,0
1153749,8f61979007ffdef4920cf2e3384e2030,Even though it was a great story (although som...,0
660066,24d04e3d0b2fe7ef8366de93618eb904,She remains one of my favourite children's aut...,0
296231,af941aae76ff1b9c9b256cbb8ebc2aef,Julius kissed him again.,0
611524,46e23d760843b8c9a4108a55cd92413d,"Or being post-TWS, except that Shield still ex...",0
178945,c66ec6d215a6d6a6ced2398ebf56ce70,"""Delicate, beautiful, fragile.",0


In [63]:
vectorizer = CountVectorizer(min_df=1000)
vectorized = vectorizer.fit_transform(all_reviews["sent"])
vectorized.shape

(2457426, 2517)

In [64]:
pro_reviews = vectorized.toarray()
new_reviews = all_reviews.to_numpy()
print(new_reviews)

users_raw_data = {}
for idx, review in enumerate(new_reviews):
    user = review[0]
    if(user not in users_raw_data ):
        users_raw_data[review[0]] = { "id": user, "x":[], "y":[], "num_samples": 0, "spoiler_count":0, "n_spoiler_count":0 }
    
    users_raw_data[user]["y"].append(review[2])
    users_raw_data[user]["x"].append(pro_reviews[idx])
        
    users_raw_data[user]["num_samples"] += 1
    if(review[2] == 1):
        users_raw_data[user]["spoiler_count"] += 1
    else:
        users_raw_data[user]["n_spoiler_count"] += 1

# print(users_raw_data["ab2fadb5c7bbe55c80406d2b3692e969"])

[['58fe4880f1c5746d7311867093747775'
  'They complain a lot and almost look stupid and incompetent.' 0]
 ['abf8fe152026f9b2ac3581c956b5782b'
  'I was drawn in to the story pretty easily and stayed fairly engaged throughout, though I will admit I got a little bored there towards the end.'
  0]
 ['77008aaf6eb579f3c87a62fd59f936fd'
  "I'm glad the writers didn't completely erase her history." 0]
 ...
 ['fd5a89a93c4c94b35b1a480eaaad2166'
  'Though I like some parts of the story there are others that seems iffy for me:'
  0]
 ['1c45a55c17e09a1a9148b8a401274fcd' 'The end!' 0]
 ['2138f2f7e5d766220af5a36e7dc0c9e1'
  'Andie has her whole summer planned out.' 0]]


In [231]:
user_list = []
user_data_train = {}
user_data_test = {}

totals_train = []
totals_test = []

random.seed(0)
np.random.seed(0)

sequence = [i for i in range(10,21)]
# sequence2 = [i for i in range(10,21)]

count = 0
# print(perm)
for user in users_raw_data:
    if(count >= 100 or users_raw_data[user]["spoiler_count"] < 3 or users_raw_data[user]["n_spoiler_count"] < 3 ):
        continue
    
    sequence = [i for i in range(1, 5 )]
    sequence2 = [i for i in range(1,  5 )]
    
    limit_spoiler = random.choice(sequence)
    limit_non    = random.choice(sequence2)
#     print(limit_non)
#     print(limit_spoiler)
    
    user_list.append(user)

    
    user_data_raw_ = users_raw_data[user]
    
    user_data = {"x":[], "y": []}
    
    for x,y in zip(user_data_raw_["x"], user_data_raw_["y"]):
        if(y == 0 and limit_non > 0 ):
            user_data["x"].append(x)
            user_data["y"].append(y)
            limit_non -= 1
        
        if(y == 1 and limit_spoiler > 0 ):
            user_data["x"].append(x)
            user_data["y"].append(y)
            limit_spoiler -= 1
            
#     print(y)
    
#     user_data["x"] = user_data["x"][:limit]
#     user_data["y"] = user_data["y"][:limit]
    
    perm = np.random.permutation(len(user_data["x"]))
    user_data["x"] = np.array(user_data["x"])
    user_data["y"] = np.array(user_data["y"])
    
    user_data["x"] = user_data["x"][perm]
    user_data["y"] = user_data["y"][perm]
        
    split = int(len(user_data["x"])*0.8)

    train_x = user_data["x"][:split]
    train_y = user_data["y"][:split]
    
    test_x  = user_data["x"][split:]
    test_y  = user_data["y"][split:]
    
    if(count < 5):
        print(perm)
        print(len(user_data["x"]))
        print(train_y)
        print(test_y)
    
    user_data_train[user] = {"y": np.array(train_y).tolist(), "x": np.array(train_x).tolist()}
    user_data_test[user] = {"y":  np.array(test_y).tolist(), "x":  np.array(test_x).tolist()}
    
    count += 1
    
    totals_train.append(len(train_y))
    totals_test.append(len(test_y))
#     print(users_raw_data[user]["num_samples"])
# print(user_list)
print(len(totals_train))
print(totals_train)
print(totals_test)


[6 2 1 3 0 5 4]
7
[1 0 1 0 0]
[1 0]
[3 0 2 1]
4
[1 0 0]
[0]
[3 1 2 4 5 0 6 7]
8
[0 0 0 1 1 0]
[1 1]
[5 2 3 4 1 0 6]
7
[1 0 0 1 0]
[0 1]
[2 3 4 1 0]
5
[1 1 1 0]
[0]
100
[5, 3, 6, 5, 4, 4, 2, 4, 3, 3, 4, 5, 4, 6, 3, 1, 4, 5, 4, 2, 3, 4, 1, 5, 3, 3, 4, 5, 4, 4, 4, 3, 3, 4, 2, 2, 4, 4, 4, 5, 5, 3, 4, 4, 2, 3, 4, 4, 4, 2, 2, 1, 2, 4, 3, 1, 2, 2, 4, 1, 4, 3, 2, 4, 4, 4, 1, 4, 4, 4, 2, 3, 4, 4, 2, 6, 4, 5, 2, 4, 3, 4, 4, 4, 4, 5, 4, 2, 4, 3, 6, 4, 6, 2, 4, 4, 6, 1, 5, 5]
[2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2]


In [232]:
train_dict = {"users": user_list, "user_data": user_data_train, "num_samples": totals_train}
test_dict = {"users": user_list, "user_data": user_data_test, "num_samples": totals_test}
# totals_train
print(len(totals_train))


100


In [233]:
import json

with open('data/train/train.json', 'w') as json_file:
    json.dump(train_dict, json_file)
with open('data/test/test.json', 'w') as json_file:
    json.dump(test_dict, json_file)